In [ ]:
%cd ..

In [ ]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(),"apperception"))

### IMPORTS
import cv2

from world import *
from world_executor import *
from video_util import *
from metadata_util import *
import json
import pandas as pd

from scenic_generate_df import *


In [ ]:
### Directly import all mini data to db without using the API
### Make sure you've downloaded the data from drive
from scenic_util import *
import psycopg2
conn = psycopg2.connect(
        database="mobilitydb", user="docker", password="docker", host="localhost", port=25432)
clean_tables(conn)
import_tables(conn)

In [ ]:
### Let's define some attribute for constructing the world first
name = 'ScenicWorld' # world name
units = 'metrics'      # world units
user_data_dir = os.path.join("v1.0-mini")

In [ ]:
# generate dataframes from scenic json files
# takes 1min to run
df_sample_data, df_annotation = scenic_generate_df()

In [ ]:
df_sample_data

In [ ]:
df_annotation

In [ ]:
scenic_world = World(name=name, units=units)

In [ ]:
### Ingest the camera to the world
scene_name = "scene-0061"
scenic_world_61 = scenic_world.camera(scene_name)

In [ ]:
### Call execute on the world to run the detection algorithm and save the real data to the database
recognized_scenic_world_61 = scenic_world_61.recognize(scene_name, df_sample_data, df_annotation)
recognized_scenic_world_61.execute()

In [ ]:
scenes = ["scene-0103","scene-0553", "scene-0655", "scene-0757", "scene-0796",
          "scene-0916", "scene-1077", "scene-1094", "scene-1100"]
for scene in scenes:
    current_scenic_world = scenic_world.camera(scene)
    current_recognized_scenic_world = current_scenic_world.recognize(scene, df_sample_data, df_annotation)
    current_recognized_scenic_world.execute()


In [ ]:
# the trajectory returned now includes the temporal info instead of just the spatio info
car_trajectories = scenic_world.predicate(lambda obj:obj.object_type == "vehicle.car").get_trajectory().execute()

In [ ]:
print(car_trajectories[0][0])

In [ ]:
frame_num = scenic_world.trajectory_to_frame_num(car_trajectories)
frame_num

In [ ]:
camera_info = []
for cur_frame_num in frame_num:
    camera_info.append(scenic_world.get_camera(scene_name, cur_frame_num))
camera_info[0]

In [ ]:
scenic_world.get_overlay_info(car_trajectories, camera_info)

In [ ]:
bboxes = scenic_world.predicate(lambda obj:obj.object_type == "vehicle.car").get_geo().execute()

In [ ]:
bboxes.shape

In [ ]:
### Save data to picke files ###
import pickle

sample_data_file = open("df_sample_data.pickle", "ab")
pickle.dump(df_sample_data, sample_data_file)                     
sample_data_file.close()

annonation_file = open("df_annotation.pickle", "ab")
pickle.dump(df_annotation, annonation_file)                     
annonation_file.close()